In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
h = pd.read_pickle('hourlies.pickle')

In [3]:
#Pick out numerical vars, ignore altimeter setting and just use station pressure
numvars = h.loc[:,['HOURLYDRYBULBTEMPF', 'HOURLYDewPointTempF',
                   'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYAltimeterSetting', 'HOURLYPrecip']]

In [4]:
#Trial before the real deal
#N = np.shape(numvars)[0]-1
#S = 1 + np.random.choice(N, int(0.50*(N)), replace=False)
#samp = numvars.iloc[S,:]

### Random Forest Build

A random forest was built to determine if a model for predicting temperatures could be built.   The tuning parameters were not fine-tuned. Cross validation was performed with 10-fold cross validation.  The train/test split was 80/20.

In [5]:
#import
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [6]:
X = numvars.drop('HOURLYDRYBULBTEMPF', axis = 1)
y = numvars['HOURLYDRYBULBTEMPF']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [8]:
forest = RandomForestRegressor(max_depth=1000, random_state=0)  
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=1000,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [9]:
forest_pred = forest.predict(X_test)

In [10]:
from sklearn.metrics import r2_score

In [11]:
test_score = r2_score(y_test, forest_pred)
test_score

0.9999014451738939

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
#Mean of 10-fold cross validation
cross_val_score(forest, X_train, y_train, cv=10)

array([0.99990229, 0.99990304, 0.99989644, 0.99990137, 0.9999025 ,
       0.99990995, 0.99989914, 0.99990723, 0.99989584, 0.99987649])

In [ ]:
## This seems too good to be true....something must be overfitting or incorrect..?
np.mean(cross_val_score(forest, X_train, y_train, cv=10))